In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

from datetime import datetime, timedelta
from pandas import DataFrame, to_datetime, set_option
from TpsParse.Tps.TpsFile import TpsFile
import os
from numpy import quantile, nan
from math import ceil
from math import pi

set_option('display.max_columns', 500)
set_option('display.max_row', 1000)

from bokeh.resources import CDN
from bokeh.embed import components
from bokeh.models import Range1d, NumeralTickFormatter, ColumnDataSource, FactorRange, HoverTool
from bokeh.plotting import figure
from bokeh.transform import linear_cmap, factor_cmap, cumsum
from bokeh.io import show, output_notebook
from bokeh.palettes import Category20_20

from django.db.models import Sum, DateField
from django.db.models.functions import Trunc
from Utils.clean_string import clean_string

output_notebook()

Loading BokehJS ...

In [2]:
def parser(path):
    file = open(path, "r+b")
    tps = TpsFile(file)

    cols_and_type = []
    columns = []
    records = []
    for definition in tps.get_table_definitions():
        for field in definition.fields:
            column = field.field_name.split(':')[1].lower()
            col_and_type = (field.field_name.split(':')[1].lower(), field.type)
            if column not in columns:
                columns.append(column)
                cols_and_type.append(col_and_type)
        for record in tps.get_data_records(definition):
            records.append([r.strip().title() if isinstance(r, str) else r for r in record.values])

    table = []
    for record in records:
        dic = {}
        for col, rec in zip(columns, record):
            dic[col] = rec
        table.append(dic)

    df = DataFrame(table, columns=columns)

    name_list = ('fec', 'periodo')
    # Formatear la fecha.
    date_columns = [cname for cname, ctype in cols_and_type if cname.startswith(name_list) and ctype == 'SignedLong']
    for col in df[date_columns]:
        df[col] = datetime(1800, 12, 28) + df[col].map(timedelta)
        df.loc[df[col] == '1800-12-28', col] = None

    # Reemplazar los campos vacíos con None.
    df.replace(to_replace='', value=nan, inplace=True)
    df.replace(to_replace=0, value=None, inplace=True)
    
    # Borrar los registros que tengan todos valores nulos.
    df.dropna(axis='index', how='all', inplace=True)
    
    # Eliminar registros duplicados.
    df.drop_duplicates(keep='last', inplace=True)
    
    # Borrar las columnas que tengan todos valores nulos o ceros.
    df.dropna(axis='columns', how='all', inplace=True)
    df = df.loc[:, (df != 0).any(axis=0)]
    
    return df

In [56]:
from app.models import Movimpu

movimpu = Movimpu.objects.values(
    'cod_imput__descripcion',
    'cod_imput__donde_acumula__descripcion',
    'fecha_comprob',
    'periodo_contable',
    'tipo_movim',
    'num_comprob',
    'descripcion',
    'importe',
    'actividad__descripcion')\
.filter(importe__gt=0)\
.order_by('periodo_contable')

df = DataFrame(movimpu)
df = df.rename({
    'actividad__descripcion': 'actividad',
    'cod_imput__descripcion': 'imputacion_simple',
    'cod_imput__donde_acumula__descripcion': 'imputacion_acumulada',
},axis='columns')
df['periodo_contable'] = to_datetime(df['periodo_contable'])
df['fecha_comprob'] = to_datetime(df['fecha_comprob'])
# Diferenciar ingresos de egresos
df['tipo'] = 'Egreso'
df.loc[(df['tipo_movim'] == 'Cobranza') | (
        df['tipo_movim'] == 'Nota De Crédito'), 'tipo'] = 'Ingreso'

In [65]:
a = df[df.imputacion_simple == 'Iva'].copy()
DataFrame(a.groupby(['actividad']).importe.sum())

,importe
actividad,


In [58]:
gb = DataFrame(df.groupby('tipo', as_index=False).agg({'importe': 'sum'}))
ingresos = gb.loc[gb.tipo == 'Ingreso'].importe
ingresos = round(float(ingresos), 2) if not ingresos.empty else None
egresos = gb.loc[gb.tipo == 'Egreso'].importe
egresos = round(float(egresos), 2) if not egresos.empty else None
print(ingresos, egresos); gb

29342268.25 103151280.3


,tipo,importe
0,Egreso,1.031513e+08
1,Ingreso,2.934227e+07


In [7]:
gb = DataFrame(df.groupby(['imputacion_acumulada','tipo'], as_index=False).importe.sum())
gb2 = DataFrame(gb.groupby('imputacion_acumulada', as_index=False).tipo.count())

for x in gb2.imputacion_acumulada:
    total = 0
    dato = gb[gb.imputacion_acumulada == x].copy()
    ingreso = dato[dato.tipo == 'Ingreso'].importe.copy()
    if not ingreso.empty:
        total += ingreso.array
    egreso = dato[dato.tipo == 'Egreso'].importe.copy()
    if not egreso.empty:
        total -= egreso.array
    gb2.loc[gb2.imputacion_acumulada == x, 'importe'] = total
gb2 = gb2.append({'imputacion_acumulada': 'aaaaaa', 'tipo': 1, 'importe': 20000000}, ignore_index=True); gb2

,imputacion_acumulada,tipo,importe
0,Administracion,2,-9761256.77
1,Alquiler Rural,1,-5949076.13
2,Combustible,1,-750838.60
3,Compra De Inmuebles,1,-85361.01
4,Creditos,1,-6180179.92
5,Ganaderia,1,-1386624.90
6,Gastos Varios,1,-207104.46
7,Honorarios,1,-434870.55
8,Impuestos,1,-2158402.67
9,Indumentaria,1,-6886.28


In [8]:
from datetime import datetime
start = datetime(2014, 4, 1)
end = datetime(2014, 4, 30)

In [9]:
df = df[(df['periodo_contable'] >= start) & (df['periodo_contable'] <= end)].copy()

In [42]:
df = df.loc[df['imputacion_acumulada'].str.contains('Insumos Agricultura', case=False, na=False)].copy()

In [29]:
DataFrame(df.groupby(['imputacion_acumulada', 'tipo']).importe.sum())

Importe
Imputación Acumulada        Tipo                 
Administracion              Egresos   38310264.15
                            Ingresos  28549007.38
Alquiler Rural              Egresos    5949076.13
Combustible                 Egresos     750838.60
Compra De Inmuebles         Egresos      85361.01
Creditos                    Egresos    6180179.92
Ganaderia                   Egresos    1386624.90
Gastos Varios               Egresos     207104.46
Honorarios                  Egresos     434870.55
Impuestos                   Egresos    2158402.67
Indumentaria                Egresos       6886.28
Insumos Agricultura         Egresos    2663645.03
                            Ingresos    217901.00
Insumos Ganaderia           Egresos     126834.53
Insumos Para Cria           Egresos      97471.78
Insumos Tambo               Egresos    5940327.37
                            Ingresos    383002.72
Intereses                   Egresos     160123.27
                            Ingresos      2999.72
Mantenimiento               Egresos    1301684.55
                            Ingresos       962.36
Publicidad                  Egresos       6566.10
Seguros                     Egresos     200496.92
                            Ingresos       270.81
Servicios                   Egresos     158080.12
Servicios Agrop Agricultura Egresos    2038879.96
Servicios Agrop Ganaderia   Egresos     340647.80
Servicios Agrop Tambo       Egresos    2769639.56
                            Ingresos    181167.70
Servicios De 3º             Egresos    1361967.32
                            Ingresos      6956.56
Venta De Granos             Egresos    8662789.34
Venta Leche                 Egresos   11324044.31
Ventas Animales             Egresos    9552366.69

In [40]:
gb_mov_impu = DataFrame(df.groupby(['tipo']).importe.sum())
gb_mov_impu.sort_values('importe', ascending=False, inplace=True); gb_mov_impu

,Importe
Tipo,
Egresos,1.031513e+08
Ingresos,2.934227e+07


In [3]:
from app.utils import get_movimpu, get_df_mov_impu
from datetime import datetime
from pandas import to_datetime

In [7]:
DataFrame(filter_date.groupby(['año', 'mes', 'tipo']).importe.sum())

importe
año  mes tipo                
2016 Aug Egresos   2415094.23
         Ingresos   498960.61
     Dec Egresos   3840897.54
         Ingresos   621378.72
     Jul Egresos   2855173.46
         Ingresos  1298320.25
     Nov Egresos   2385625.44
         Ingresos   569881.51
     Oct Egresos   2195569.87
         Ingresos  1052607.61
     Sep Egresos   2510449.03
         Ingresos   913505.28
2017 Apr Egresos   2690394.33
         Ingresos  1023384.40
     Feb Egresos   3040151.24
         Ingresos   433881.60
     Jan Egresos   2500902.84
         Ingresos  1613504.00
     Jun Egresos   1654438.52
         Ingresos   484389.87
     Mar Egresos   1737927.20
         Ingresos   423437.00
     May Egresos   4556341.00
         Ingresos  1022027.10

In [9]:
filter_date[filter_date['nro_mes'] == 7]

,imputacion_simple,cod_imput__donde_acumula,imputacion_acumulada,fecha_comprob,periodo_contable,tipo_movim,num_comprob,descripcion,importe,actividad,provincia,tipo_proveedor,tipo,nro_mes,mes,año
7299,Imp Ley 25413 Afip,19,Impuestos,2016-07-01,2016-07-01,Documento De Contado,3344,Afip,825.13,None,None,None,Egresos,7,Jul,2016
7300,Seguros Accidentes Personales,24,Seguros,2016-06-10,2016-07-01,Facturas Cuenta Corriente,2401618,Sancor Seguros,166.95,None,None,Seguros,Egresos,7,Jul,2016
7301,Seguros Accidentes Personales,24,Seguros,2016-06-30,2016-07-01,Facturas Cuenta Corriente,239656,Sancor Seguros,1326.52,None,None,Seguros,Egresos,7,Jul,2016
7302,Nafta,12,Combustible,2016-06-26,2016-07-01,Facturas Cuenta Corriente,88683,Hugo L.A. Mancini,341.91,None,Catamarca,Combustible,Egresos,7,Jul,2016
7303,Seguros Accidentes Personales,24,Seguros,2016-06-04,2016-07-01,Facturas Cuenta Corriente,680565,Sancor Seguros,21.9,None,None,Seguros,Egresos,7,Jul,2016
7304,Comision Mant. De Cuenta,19,Impuestos,2016-07-01,2016-07-01,Documento De Contado,3289,Comis Manten Cuenta,369.6,None,None,None,Egresos,7,Jul,2016
7305,Control Lechero,8,Servicios De 3º,2016-06-22,2016-07-01,Facturas Cuenta Corriente,439,Dante J.P. Grangetto,2120,None,None,Mantenimientos Agropec,Egresos,7,Jul,2016
7306,Honorarios Contables,21,Honorarios,2016-06-02,2016-07-01,Facturas Cuenta Corriente,2062016,Dr Oscar F.J. Canello,1391.5,None,None,Honorarios Administracion,Egresos,7,Jul,2016
7307,Uatre,19,Impuestos,2016-06-02,2016-07-01,Facturas Cuenta Corriente,2062016,Dr Oscar F.J. Canello,562.88,None,None,Honorarios Administracion,Egresos,7,Jul,2016
7308,Honorarios Contables,21,Honorarios,2016-06-02,2016-07-01,Facturas Cuenta Corriente,802,Dr Oscar F.J. Canello,2150,None,None,Honorarios Administracion,Egresos,7,Jul,2016


In [5]:
movimpu = get_movimpu()
df = get_df_mov_impu(movimpu)
df.loc[:, 'periodo_contable'] = to_datetime(df['periodo_contable'])

def mapper(month):
   return month.strftime('%b')

inicio = datetime(2016,7,1)
fin = datetime(2017,7,1)
filter_date = df[(df['periodo_contable'] >= inicio) & (df['periodo_contable'] < fin)]
filter_date.loc[:, 'nro_mes'] = filter_date['periodo_contable'].dt.month
filter_date.loc[:, 'mes'] = filter_date['periodo_contable'].apply(mapper)
filter_date.loc[:, 'año'] = filter_date['periodo_contable'].dt.year.astype('str')
gb = DataFrame(filter_date.groupby(['año', 'mes', 'nro_mes'], as_index=False).importe.sum()).sort_values(['año','nro_mes'])
gb = gb.set_index(['año', 'mes'])
source = ColumnDataSource(gb)

ingresos = filter_date[filter_date['tipo'] == 'Ingresos'].copy()
egresos = filter_date[filter_date['tipo'] == 'Egresos'].copy()

gb_ingresos = DataFrame(ingresos.groupby(['año', 'mes', 'nro_mes'], as_index=False).importe.sum()).sort_values(['año','nro_mes'])
gb_ingresos = gb_ingresos.set_index(['año', 'mes'])
source_ingresos = ColumnDataSource(gb_ingresos)
gb_egresos = DataFrame(egresos.groupby(['año', 'mes', 'nro_mes'], as_index=False).importe.sum()).sort_values(['año','nro_mes'])
gb_egresos = gb_egresos.set_index(['año', 'mes'])
source_egresos = ColumnDataSource(gb_egresos)

plot = figure(
    x_range=FactorRange(*source.data['año_mes'])
)
plot.line(x='año_mes', y='importe', source=source_ingresos, line_color='green')
plot.circle(x='año_mes', y='importe', source=source_ingresos, fill_color=None)
plot.line(x='año_mes', y='importe', source=source_egresos, line_color='red')
plot.circle(x='año_mes', y='importe', source=source_egresos, fill_color=None)
plot.yaxis.formatter = NumeralTickFormatter(format="$0")
plot.sizing_mode = 'stretch_width'

show(plot)